# Download data from Google Earth Engine

The following notebook is an example to download data from Google Earth Engine (GEE). This is a copy from the original, some code has change to be in acordance with the most recent libraries changes and the changes in GEE. 

This should be run on a jupyterlab, the use of this notebook on something like visualstudio code is not granted, the Authenticate by Earth Engine is not posible since the output of the cell somethimes can't been seen. This error is not constant an is time consuming to try to fixed therefore the use of jupyterlab is preferd. 

In [ ]:
import ee
import geemap
import folium
import os 
import rasterio
import numpy as np
import requests 
import zipfile
import time

In [ ]:
ee.Authenticate()

True

In [ ]:
ee.Initialize()


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [ ]:
#| export
def humedad_relativa(image):
    # Se utiliza la siguiente formula 
      # e = 6.1078 *math.exp((17.1*Tem_dew)/(235+Tem_dew) )
      # e_s = 6.1078 *math.exp((17.1*Temp)/(235+Temp) )
      # HR = (e/e_s)*100
   
    lambda_i = 240.97
    beta_i = 17.502 #### #17.1; #17.625;
    coef= 611.21; ####// reference level water vapor pressure at 273.15 K
    dew_point =  image.select('dewpoint_temperature_2m')
    temp_2m =  image.select('temperature_2m')
    evap = image.select('total_evaporation')
    tot_pre_h = image.select('total_precipitation_hourly')
    tot_pre = image.select('total_precipitation')
    e_up  =  dew_point.multiply(beta_i)
    e_dow =   dew_point.add(lambda_i)
    e = ((e_up.divide(e_dow)).exp()).multiply(coef)
    e_s_up  =  temp_2m.multiply(beta_i)
    e_s_dow =   temp_2m.add(lambda_i);
    e_s   = ((e_s_up.divide(e_s_dow)).exp()).multiply(coef);
    rel_hum = (e.divide(e_s)).multiply(100);
    
    rel_hum = rel_hum.select(['dewpoint_temperature_2m']).rename(['relative_humidity']);
      
    evap_mul = evap.multiply(lambda_i);
    tot_pre_h_mul = tot_pre_h.multiply(beta_i);
    tot_pre_mul =   tot_pre.multiply(coef);
    tonteria = (evap_mul.add(tot_pre_h_mul)).add(tot_pre_mul)
    tonteria = tonteria.select(['total_evaporation']).rename(['tonteria']);
    rel_hum.select(['dewpoint_temperature_2m']).rename(['relative_humidity']);
    
    image = image.addBands([rel_hum, tonteria]);
    
    return image


In [ ]:
def  get_image_raster_mean(collection, start_date, end_date):
    
    
    numberOfDays = end_date.difference(start_date, 'days')
    lapse = ee.List.sequence(0, numberOfDays.subtract(1)) 
    def next_day(day):
        return ee.Date(start_date).advance(day, 'day')
    
    listdates = lapse.map(next_day)
    def mean_day(day):
        day_collection = collection.filter(ee.Filter.date(day,ee.Date(day).advance(1, 'day') ))
        mean_day=  day_collection.mean();
        mean_day= mean_day.set('day' , day)
        return mean_day
    
    image_means = listdates.map(mean_day)
    
    return ee.ImageCollection(image_means);



In [ ]:

def ima_land_reproject (image_col, reproject_pro ):
    return ee.Image(image_col).resample('bilinear').reproject(crs= reproject_pro, scale=926.625 )
    #return ee.Image(image_col).reproject(crs= proj_, scale=926.625 )

    
    
def ima_land_reproject_no_resample(image_col, reproject_pro ):
    return ee.Image(image_col).reproject(crs= reproject_pro, scale=926.625 )
    #return ee.Image(image_col).reproject(crs= proj_, scale=926.625 )
    


In [ ]:
def get_geo_tif(image,
                file_name_tif,
                unmask_val = -100, 
                url_params = None,
                re_fill_nan = True,
                timeout=300):
    """Save the image as geotiff"""
    
    if url_params is None:
        url_params =  {
            'bands': [
              "dewpoint_temperature_2m",
              "total_evaporation",
              "temperature_2m",
              "total_precipitation_hourly",
              "total_precipitation",
              "relative_humidity",
              "Optical_Depth_047",
              "AOD_Uncertainty"
            ],
            'format': "GEO_TIFF"
            }
    
    try:
        url_image = image.unmask(unmask_val).getDownloadUrl(url_params)
    except Exception as e:
        print("An error occurred while downloading.")
        print(e)
        return
    print('getting url: \n', url_image)
    r= requests.get(url_image, stream=True, timeout=timeout)
    if r.status_code != 200:
        print("An error occurred while downloading.")
        return
    
    print('Saving at : \n', file_name_tif)
    with open(file_name_tif, "wb") as fd:
        for chunk in r.iter_content(chunk_size=1024):
            fd.write(chunk)
        fd.close()
    
    
    if re_fill_nan:
        print("Fill values with Nan")
        bands_change_nan(file_name_tif)
    
    
    return 


In [ ]:
def get_geo_collection(gee_collection, 
                       path, 
                       file_names, 
                       unmask_value= -100,
                       re_fill_nan= True,
                       timeout=300,
                       ):
    """Funcion to get all the collection"""
    count = int(gee_collection.size().getInfo())

    if file_names is None:
        file_names = ee_object.aggregate_array("system:index").getInfo()
        
    elif isinstance(file_names, int):
        file_names = [str(f + file_names) for f in range(0, count)]
    
    if len(file_names) != count:
        raise Exception(
            "The number of filenames must be equal to the number of images."
        )
    file_names = [str(f) + ".tif" for f in file_names if not str(f).endswith(".tif")]
    
    for i in range(0, count):
        image = ee.Image(gee_collection.toList(count).get(i))
        filename = os.path.join(path, file_names[i])
        print(f"Exporting {i + 1}/{count}: {filename}")
        get_geo_tif(
            image,
            file_name_tif = filename,
            unmask_val = unmask_value,
            re_fill_nan = re_fill_nan,
            timeout = timeout
        )
        print("\n")


The use of the GEE collection **'MODIS/061/MCD19A2_GRANULES'** is use insted of **MODIS/006/MCD19A2_GRANULES** this is becauuse the secon one has been discontinued. 

In [ ]:
#| export 
def get_image_range(start_Date, end_Date, get_zm ):
    """
    Get a image collection with AOD and meteorogical data on the images
    """
    
    range_all_images = ee.DateRange(start_Date, end_Date)# Rango de dias
    difference_days = range_all_images.end().difference(range_all_images.start() , 'day')
    #todos los dias como una secuencia
    
    bb_get_zm = get_zm.geometry().bounds()
    #print(bb_get_zm.getInfo())
    def clip_zm_fun(img):
        return img.clip(bb_get_zm)
    
    land_eras_day_f = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filter(
            ee.Filter.date(start_Date, end_Date)
        ).select([
            'dewpoint_temperature_2m',
            'total_evaporation',
            'temperature_2m',
            'total_precipitation_hourly',
            'total_precipitation'
        ]).map(clip_zm_fun)
    
    AOD_MODIS_day_f =  ee.ImageCollection("MODIS/061/MCD19A2_GRANULES").filter(
        ee.Filter.date(start_Date, end_Date)
        ).select([
            'Optical_Depth_047',
            'AOD_Uncertainty'
        ]).map(clip_zm_fun)
    
    proje_pro = get_zm.first().geometry().projection()
    
    def ima_land_reproject_wrap(image):
        return ima_land_reproject(image,proje_pro)
    
    def ima_land_no_reproject_wrap(image):
        return ima_land_reproject_no_resample(image,proje_pro)
    
    
    land_eras_day_f = land_eras_day_f.map(humedad_relativa)
    land_eras_day_f = get_image_raster_mean(land_eras_day_f, start_Date, end_Date)
    land_eras_day_f = land_eras_day_f.map(ima_land_reproject_wrap)
    
    AOD_MODIS_day_f = get_image_raster_mean(AOD_MODIS_day_f,start_Date, end_Date)
    AOD_MODIS_day_f = AOD_MODIS_day_f.map(ima_land_no_reproject_wrap)
    
    
    
    
    
    ##### Se unen las collecciones 
    filter_maps = ee.Filter.equals(leftField= 'day',rightField= 'day')
    simpleJoin = ee.Join.inner()
    innerJoin = ee.ImageCollection(simpleJoin.apply(land_eras_day_f, AOD_MODIS_day_f, filter_maps))
    
    def cat_fun(feature):
        return ee.Image.cat(feature.get('primary'), feature.get('secondary'))
    
    
    joined_aod_tem = innerJoin.map(cat_fun)
    joined_aod_tem = joined_aod_tem.map(clip_zm_fun)
    joined_aod_tem = joined_aod_tem.select([
                  "dewpoint_temperature_2m",
                  "total_evaporation",
                  "temperature_2m",
                  "total_precipitation_hourly",
                  "total_precipitation",
                  "relative_humidity",
                  "Optical_Depth_047",
                  "AOD_Uncertainty", 
                  "tonteria"
                ])
    return joined_aod_tem


In [ ]:
def bands_change_nan(file_path):
    """The function chenche the value of mask for np.nan"""
    src = rasterio.open(file_path, 'r+')
    
    for band_num in range(1,src.meta['count'] +1 ):
        band_arr = src.read(band_num)
        band_arr[band_arr==-100] = np.nan
        src.write(band_arr,band_num)
    src.close()    



In [ ]:
#| export 
def get_image_range_no_resample(start_Date, end_Date, get_zm ):
    """
    Get a image collection with AOD and meteorogical data on the images
    """
    
    range_all_images = ee.DateRange(start_Date, end_Date)# Rango de dias
    difference_days = range_all_images.end().difference(range_all_images.start() , 'day')
    #todos los dias como una secuencia
    
    bb_get_zm = get_zm.geometry().bounds()
    #print(bb_get_zm.getInfo())
    def clip_zm_fun(img):
        return img.clip(bb_get_zm)
    
    land_eras_day_f = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filter(
            ee.Filter.date(start_Date, end_Date)
        ).select([
            'dewpoint_temperature_2m',
            'total_evaporation',
            'temperature_2m',
            'total_precipitation_hourly',
            'total_precipitation'
        ]).map(clip_zm_fun)
    
    AOD_MODIS_day_f =  ee.ImageCollection("MODIS/006/MCD19A2_GRANULES").filter(
        ee.Filter.date(start_Date, end_Date)
        ).select([
            'Optical_Depth_047',
            'AOD_Uncertainty'
        ]).map(clip_zm_fun)
    
    proje_pro = get_zm.first().geometry().projection()
    
    def ima_land_reproject_wrap(image):
        return ima_land_reproject(image,proje_pro)
    
        return ima_land_reproject_no_resample(image,proje_pro)
    
    
    land_eras_day_f = land_eras_day_f.map(humedad_relativa)
    land_eras_day_f = get_image_raster_mean(land_eras_day_f, start_Date, end_Date)
    land_eras_day_f = land_eras_day_f.map(ima_land_no_reproject_wrap)
    
    AOD_MODIS_day_f = get_image_raster_mean(AOD_MODIS_day_f,start_Date, end_Date)
    AOD_MODIS_day_f = AOD_MODIS_day_f.map(ima_land_no_reproject_wrap)
    
    
    
    
    
    ##### Se unen las collecciones 
    filter_maps = ee.Filter.equals(leftField= 'day',rightField= 'day')
    simpleJoin = ee.Join.inner()
    innerJoin = ee.ImageCollection(simpleJoin.apply(land_eras_day_f, AOD_MODIS_day_f, filter_maps))
    
    def cat_fun(feature):
        return ee.Image.cat(feature.get('primary'), feature.get('secondary'))
    
    
    joined_aod_tem = innerJoin.map(cat_fun)
    joined_aod_tem = joined_aod_tem.map(clip_zm_fun)
    joined_aod_tem = joined_aod_tem.select([
                  "dewpoint_temperature_2m",
                  "total_evaporation",
                  "temperature_2m",
                  "total_precipitation_hourly",
                  "total_precipitation",
                  "relative_humidity",
                  "Optical_Depth_047",
                  "AOD_Uncertainty", 
                  "tonteria"
                ])
    return joined_aod_tem

In [ ]:
dic_met_area = {
                    'ZMVM': "users/msalazar_cgeo/ZMVM",
                    'ZMG': "users/msalazar_cgeo/ZMG",
                    'ZMVT':"users/msalazar_cgeo/ZMVT",
                    'AMM': "users/msalazar_cgeo/AMM"}
ZM_table = ee.FeatureCollection(dic_met_area['ZMVM'])
startdate_fun = ee.Date("2004-01-21" )
enddate_fun = ee.Date("2004-01-22")
merge_join_col = get_image_range(startdate_fun, enddate_fun, ZM_table)

In [ ]:
def get_geo_collection(gee_collection, 
                       path, 
                       file_names, 
                       unmask_value= -100,
                       re_fill_nan= True,
                       timeout=300,
                       ):
    """Funcion to get all the collection"""
    count = int(gee_collection.size().getInfo())

    if file_names is None:
        file_names = ee_object.aggregate_array("system:index").getInfo()
        
    elif isinstance(file_names, int):
        file_names = [str(f + file_names) for f in range(0, count)]
    
    if len(file_names) != count:
        raise Exception(
            "The number of filenames must be equal to the number of images."
        )
    file_names = [str(f) + ".tif" for f in file_names if not str(f).endswith(".tif")]
    
    for i in range(0, count):
        image = ee.Image(gee_collection.toList(count).get(i))
        filename = os.path.join(path, file_names[i])
        print(f"Exporting {i + 1}/{count}: {filename}")
        get_geo_tif(
            image,
            file_name_tif = filename,
            unmask_val = unmask_value,
            re_fill_nan = re_fill_nan,
            timeout = timeout
        )
        print("\n")


Once the data can be downloaded we have the issu that the bands names are not saved, this can be fix with a proper function to change them, once all the data is downloaded. 

Now a full function is made to download a fix year, for 




In [ ]:
def get_year_zm(
        date_start_str,
        date_end_str,
        str_ZM,
        path_save,
        prefix = "aod_cli_"):
    
    """ The function will get all the data from the metropolitan area between
    the interval datess
    """
    dic_met_area = {
                    'ZMVM': "users/msalazar_cgeo/ZMVM",
                    'ZMG': "users/msalazar_cgeo/ZMG",
                    'ZMVT':"users/msalazar_cgeo/ZMVT",
                    'AMM': "users/msalazar_cgeo/AMM"}
    ZM_table = ee.FeatureCollection(dic_met_area[str_ZM])
    
    
    startdate_fun = ee.Date(date_start_str)
    enddate_fun = ee.Date(date_end_str)
    
    merge_join_col = get_image_range(startdate_fun, enddate_fun, ZM_table)
    
    list_dates = merge_join_col.aggregate_array('day').getInfo()

    name_dates_str = [ee.Date(i['value']).format('YYYY_MM_dd').getInfo() for i in list_dates]
    names_dates_str  = [prefix + str_ZM+ '_' + na for na in name_dates_str]
    print(names_dates_str)
    get_geo_collection(
        merge_join_col,
        path = path_save,
        file_names= names_dates_str,
        unmask_value= -100
    )
    
    return 

In [ ]:
def get_year_zm_no_resample(
        date_start_str,
        date_end_str,
        str_ZM,
        path_save,
        prefix = "aod_cli_"):
    
    """ The function will get all the data from the metropolitan area between
    the interval dates
    """
    dic_met_area = {
                    'ZMVM': "users/msalazar_cgeo/ZMVM",
                    'ZMG': "users/msalazar_cgeo/ZMG",
                    'ZMVT':"users/msalazar_cgeo/ZMVT",
                    'AMM': "users/msalazar_cgeo/AMM"}
    ZM_table = ee.FeatureCollection(dic_met_area[str_ZM])
    
    
    startdate_fun = ee.Date(date_start_str)
    enddate_fun = ee.Date(date_end_str)
    
    merge_join_col = get_image_range_no_resample(startdate_fun, enddate_fun, ZM_table)
    
    list_dates = merge_join_col.aggregate_array('day').getInfo()

    name_dates_str = [ee.Date(i['value']).format('YYYY_MM_dd').getInfo() for i in list_dates]
    names_dates_str  = [prefix + str_ZM+ '_' + na for na in name_dates_str]
    print(names_dates_str)
    get_geo_collection(
        merge_join_col,
        path = path_save,
        file_names= names_dates_str,
        unmask_value= -100
    )
    
    return 

This is how the function is call to get the data from GEE

In [ ]:
get_year_zm(
        "2022-01-01",
        "2022-03-01",
        "ZMVM",
        "../data/raster/AOD_REA_GEE/ZMVM/2022/",
        prefix = "aod_cli_")

['aod_cli_ZMVM_2022_01_01', 'aod_cli_ZMVM_2022_01_02', 'aod_cli_ZMVM_2022_01_03', 'aod_cli_ZMVM_2022_01_04', 'aod_cli_ZMVM_2022_01_05', 'aod_cli_ZMVM_2022_01_06', 'aod_cli_ZMVM_2022_01_07', 'aod_cli_ZMVM_2022_01_08', 'aod_cli_ZMVM_2022_01_09', 'aod_cli_ZMVM_2022_01_10', 'aod_cli_ZMVM_2022_01_11', 'aod_cli_ZMVM_2022_01_12', 'aod_cli_ZMVM_2022_01_13', 'aod_cli_ZMVM_2022_01_14', 'aod_cli_ZMVM_2022_01_15', 'aod_cli_ZMVM_2022_01_16', 'aod_cli_ZMVM_2022_01_17', 'aod_cli_ZMVM_2022_01_18', 'aod_cli_ZMVM_2022_01_19', 'aod_cli_ZMVM_2022_01_20', 'aod_cli_ZMVM_2022_01_21', 'aod_cli_ZMVM_2022_01_22', 'aod_cli_ZMVM_2022_01_23', 'aod_cli_ZMVM_2022_01_24', 'aod_cli_ZMVM_2022_01_25', 'aod_cli_ZMVM_2022_01_26', 'aod_cli_ZMVM_2022_01_27', 'aod_cli_ZMVM_2022_01_28', 'aod_cli_ZMVM_2022_01_29', 'aod_cli_ZMVM_2022_01_30', 'aod_cli_ZMVM_2022_01_31', 'aod_cli_ZMVM_2022_02_01', 'aod_cli_ZMVM_2022_02_02', 'aod_cli_ZMVM_2022_02_03', 'aod_cli_ZMVM_2022_02_04', 'aod_cli_ZMVM_2022_02_05', 'aod_cli_ZMVM_2022_02_06', 

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/2a8989ae2a968a6e912082af211db18d-9fb9b19b14d78557e73c9e8522c3d22a:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_01_02.tif
Fill values with Nan


Exporting 3/59: ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_01_03.tif


Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/f01e97f7d39e3e5da7bd15f380b9041c-6dede571f916de94d3b7f08971957ec8:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_01_03.tif
Fill values with Nan


Exporting 4/59: ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_01_04.tif


...

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/d8c5ac35155582009ddf90293075b697-3f65b57a6451f3d3de8c9d215df0f3d6:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_02_27.tif
Fill values with Nan


Exporting 59/59: ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_02_28.tif


Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/9caf568165af214ff3a51bcae0f68e9a-3d85d01996044e62eef77e890b7b51e4:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_02_28.tif
Fill values with Nan




Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


In [ ]:
get_year_zm(
        "2022-03-01",
        "2022-08-01",
        "ZMVM",
        "../data/raster/AOD_REA_GEE/ZMVM/2022/",
        prefix = "aod_cli_")

['aod_cli_ZMVM_2022_03_01', 'aod_cli_ZMVM_2022_03_02', 'aod_cli_ZMVM_2022_03_03', 'aod_cli_ZMVM_2022_03_04', 'aod_cli_ZMVM_2022_03_05', 'aod_cli_ZMVM_2022_03_06', 'aod_cli_ZMVM_2022_03_07', 'aod_cli_ZMVM_2022_03_08', 'aod_cli_ZMVM_2022_03_09', 'aod_cli_ZMVM_2022_03_10', 'aod_cli_ZMVM_2022_03_11', 'aod_cli_ZMVM_2022_03_12', 'aod_cli_ZMVM_2022_03_13', 'aod_cli_ZMVM_2022_03_14', 'aod_cli_ZMVM_2022_03_15', 'aod_cli_ZMVM_2022_03_16', 'aod_cli_ZMVM_2022_03_17', 'aod_cli_ZMVM_2022_03_18', 'aod_cli_ZMVM_2022_03_19', 'aod_cli_ZMVM_2022_03_20', 'aod_cli_ZMVM_2022_03_21', 'aod_cli_ZMVM_2022_03_22', 'aod_cli_ZMVM_2022_03_23', 'aod_cli_ZMVM_2022_03_24', 'aod_cli_ZMVM_2022_03_25', 'aod_cli_ZMVM_2022_03_26', 'aod_cli_ZMVM_2022_03_27', 'aod_cli_ZMVM_2022_03_28', 'aod_cli_ZMVM_2022_03_29', 'aod_cli_ZMVM_2022_03_30', 'aod_cli_ZMVM_2022_03_31', 'aod_cli_ZMVM_2022_04_01', 'aod_cli_ZMVM_2022_04_02', 'aod_cli_ZMVM_2022_04_03', 'aod_cli_ZMVM_2022_04_04', 'aod_cli_ZMVM_2022_04_05', 'aod_cli_ZMVM_2022_04_06', 

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/fa97db1f446643dd09abb1e5bd8ee6b1-740f41a60524e0748f63350db28776e2:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_03_02.tif
Fill values with Nan


Exporting 3/153: ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_03_03.tif


Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/0380ede8f57ab444a4811094329495f4-f500b5b92bb20c27c1e0e815f0964ece:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_03_03.tif
Fill values with Nan


Exporting 4/153: ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_03_04.tif


...

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/44325d0199371eaa4d00e49bed66e9f1-912f9071084cd1be2f90772b92a092b1:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_07_30.tif
Fill values with Nan


Exporting 153/153: ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_07_31.tif


Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/3859e53eed3e909f7f1c37f1e99d64c2-d5c4d6e2fbd1130f9c01d137faa82f9a:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_07_31.tif
Fill values with Nan




Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


In [ ]:
get_year_zm(
        "2022-08-01",
        "2023-01-01",
        "ZMVM",
        "../data/raster/AOD_REA_GEE/ZMVM/2022/",
        prefix = "aod_cli_")

['aod_cli_ZMVM_2022_08_01', 'aod_cli_ZMVM_2022_08_02', 'aod_cli_ZMVM_2022_08_03', 'aod_cli_ZMVM_2022_08_04', 'aod_cli_ZMVM_2022_08_05', 'aod_cli_ZMVM_2022_08_06', 'aod_cli_ZMVM_2022_08_07', 'aod_cli_ZMVM_2022_08_08', 'aod_cli_ZMVM_2022_08_09', 'aod_cli_ZMVM_2022_08_10', 'aod_cli_ZMVM_2022_08_11', 'aod_cli_ZMVM_2022_08_12', 'aod_cli_ZMVM_2022_08_13', 'aod_cli_ZMVM_2022_08_14', 'aod_cli_ZMVM_2022_08_15', 'aod_cli_ZMVM_2022_08_16', 'aod_cli_ZMVM_2022_08_17', 'aod_cli_ZMVM_2022_08_18', 'aod_cli_ZMVM_2022_08_19', 'aod_cli_ZMVM_2022_08_20', 'aod_cli_ZMVM_2022_08_21', 'aod_cli_ZMVM_2022_08_22', 'aod_cli_ZMVM_2022_08_23', 'aod_cli_ZMVM_2022_08_24', 'aod_cli_ZMVM_2022_08_25', 'aod_cli_ZMVM_2022_08_26', 'aod_cli_ZMVM_2022_08_27', 'aod_cli_ZMVM_2022_08_28', 'aod_cli_ZMVM_2022_08_29', 'aod_cli_ZMVM_2022_08_30', 'aod_cli_ZMVM_2022_08_31', 'aod_cli_ZMVM_2022_09_01', 'aod_cli_ZMVM_2022_09_02', 'aod_cli_ZMVM_2022_09_03', 'aod_cli_ZMVM_2022_09_04', 'aod_cli_ZMVM_2022_09_05', 'aod_cli_ZMVM_2022_09_06', 

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/8ea6c32218495bf9c2bce0ab103c498c-8990cf333a552aaabefc7911e3e5a736:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_08_02.tif
Fill values with Nan


Exporting 3/153: ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_08_03.tif


Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/cfdc7ebeabd48f760986ea7e4b38274b-7e2583ea6615800c06be7f40b74a116f:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_08_03.tif
Fill values with Nan


Exporting 4/153: ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_08_04.tif


...

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/6e55fadd9a7f68fba0b0640080cf428e-c3caf9297a7bca20a3c3eddea80f4de7:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_12_30.tif
Fill values with Nan


Exporting 153/153: ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_12_31.tif


Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


getting url: 
 https://earthengine.googleapis.com/v1/projects/851206260265/thumbnails/d6202823f1d6beb4bc4a7b9ec0419413-0d2c175902accfcdc4399872b61ae075:getPixels
Saving at : 
 ../data/raster/AOD_REA_GEE/ZMVM/2022/aod_cli_ZMVM_2022_12_31.tif
Fill values with Nan




Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


In [ ]:
import rasterio
def set_band_names_gee(file_path):
    """Rename the bands in the file """
    dst_m= rasterio.open(file_path,'r+',  driver =  'GTiff')
    
    band_names = ["dewpoint_temperature_2m",
                  "total_evaporation",
                  "temperature_2m",
                  "total_precipitation_hourly",
                  "total_precipitation",
                  "relative_humidity",
                  "Optical_Depth_047",
                  "AOD_Uncertainty"]
    for num, band in enumerate(band_names ):
        dst_m.set_band_description(num+1, band)
    dst_m.close()


In [ ]:

def set_band_dir_gee(path_dir):
    files_li = os.listdir(path_dir)
    files_li = [fil for fil in files_li if fil[-4:] == ".tif" ]
    #files_li = [path_dir + '/'+fil for file in files_li]
    print("Changing file band names:")
    for fil in  files_li:
        set_band_names_gee(path_dir + '/'+fil)
    return
    

In [ ]:
set_band_dir_gee('../data/raster/AOD_REA_GEE/ZMVM/2022/')

Changing file band names:


Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, 